In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import bossdata.meta
import ghosts

In [11]:
def low_z_diff(table):
    """
    Return dictionary of fibers between a pair with Z-diff <= 0.001
    
    Args:
        table(str): name of bossdata.meta.Database query table with PLATE,MJD,FIBER,Z,ZWARNING
    
    Returns:
        result({str:float}): dict with 'plate-mjd-fiber' as key, z-diff as value
    """
    result = {}
    for i in range(len(table)-1):
        if table[i]['PLATE'] == table[i+1]['PLATE']:
            if table[i]['FIBER'] == table[i+1]['FIBER'] - 2:
                diff = table[i]['Z'] - table[i+1]['Z']
                fib = table[i]['FIBER']+1
                if abs(diff) <= 0.001:
                    f_str = "{}-{}-{}".format(table[i]['PLATE'],table[i]['MJD'],fib)
                    result[f_str]={'DIFF':diff,
                                   'LOW':{'Z':table[i]['Z'],'ZWARNING':table[i]['ZWARNING']},
                                   'HIGH':{'Z':table[i+1]['Z'],'ZWARNING':table[i+1]['ZWARNING']}}
    for i in range(len(table)-2):
        if table[i]['PLATE'] == table[i+2]['PLATE']:
            if table[i]['FIBER'] == table[i+2]['FIBER'] - 2:
                diff = table[i]['Z'] - table[i+2]['Z']
                fib = table[i]['FIBER']+1
                if abs(diff) <= 0.001:
                    f_str = "{}-{}-{}".format(table[i]['PLATE'],table[i]['MJD'],fib)
                    result[f_str]={'DIFF':diff,
                                   'LOW':{'Z':table[i]['Z'],'ZWARNING':table[i]['ZWARNING']},
                                   'HIGH':{'Z':table[i+2]['Z'],'ZWARNING':table[i+2]['ZWARNING']}}
    return result

In [8]:
def valid(z_dict):
    valid = 0
    invalid = 0
    for z_diff in z_dict.keys():
        fib = int(z_diff.split('-')[2])
        if fib == 500 or fib == 501:
            invalid += 1.0
        else:
            valid += 1.0
    val = valid / len(z_dict)
    inval = invalid / len(z_dict)
    print('Valid: {} -- {:.2f}%\nInvalid: {} -- {:.2f}%'.format(int(valid), val*100, int(invalid), inval*100))

In [4]:
obj_data = bossdata.meta.Database()

In [9]:
data_table = obj_data.select_all(what = 'PLATE,MJD,FIBER,Z,ZWARNING',
                                 where = 'MJD between 55170 and 55370 and Z > 0.1',
                                 max_rows = 0)

In [12]:
x= low_z_diff(data_table)

In [7]:
valid(x)

Valid: 3246 -- 99.88%
Invalid: 4 -- 0.12%


In [8]:
def warning(z_dict):
    valid = 0
    half = 0
    invalid = 0
    for stuff in z_dict.values():
        if stuff['LOW']['ZWARNING'] == 0 and stuff['HIGH']['ZWARNING'] == 0:
            valid += 1
        elif stuff['LOW']['ZWARNING'] != 0 and stuff['HIGH']['ZWARNING'] != 0:
            invalid += 1
        else:
            half += 1
    s = "No warning: {} \nWarning: {} ({} with warning on both)"
    print(s.format(valid,half+invalid,invalid))

In [9]:
warning(x)

No warning: 3007 


In [13]:
def print_warnings(zw):
    if zw == 0:
        print('No warning')
        return
    if zw & 2**0 != 0:
        print('Sky fiber')
    if zw & 2**1 != 0:
        print('Too little wavelength coverage')
    if zw & 2**2 != 0:
        print('Small delta Chi^2')
    if zw & 2**3 != 0:
        print('Negative synthetic spectrum')
    if zw & 2**4 != 0:
        print('Too many points 5+ sigma from best model')
    if zw & 2**5 != 0:
        print('Chi^2 minimum at edge of fitting range')
    if zw & 2**6 != 0:
        print('Negative emission (QSO only)')
    if zw & 2**7 != 0:
        print('Unplugged fiber')
    if zw & 2**8 != 0:
        print('Catastrophically bad data')
    if zw & 2**9 != 0:
        print('No data')

In [14]:
test = x.keys()[50:70]
for pmf in test:
    plate,mjd,fiber = pmf.split('-')
    plate = int(plate)
    mjd = int(mjd)
    fiber = int(fiber)
    value = x[pmf]
    
    print(pmf)
    print('Z-difference: {}'.format(value['DIFF']))
    print('Z value for fiber {}: {}'.format(fiber - 1, value['LOW']['Z']))
    print('Z value for fiber {}: {}'.format(fiber + 1, value['HIGH']['Z']))
    print('Warnings for fiber {}:'.format(fiber - 1))
    print_warnings(value['LOW']['ZWARNING'])
    print('Warnings for fiber {}:'.format(fiber + 1))
    print_warnings(value['HIGH']['ZWARNING'])
    
    print('')

3861-55274-600
Z-difference: -0.00045
Z value for fiber 599: 0.54153
Z value for fiber 601: 0.54198
Warnings for fiber 599:
No warning
Warnings for fiber 601:
No warning

4075-55352-226
Z-difference: -0.00063
Z value for fiber 225: 0.43519
Z value for fiber 227: 0.43582
Warnings for fiber 225:
No warning
Warnings for fiber 227:
No warning

3942-55338-391
Z-difference: 0.00024
Z value for fiber 390: 0.57367
Z value for fiber 392: 0.57343
Warnings for fiber 390:
No warning
Warnings for fiber 392:
No warning

3853-55268-88
Z-difference: 0.00061
Z value for fiber 87: 0.56736
Z value for fiber 89: 0.56675
Warnings for fiber 87:
No warning
Warnings for fiber 89:
No warning

3929-55335-676
Z-difference: 0.0009
Z value for fiber 675: 0.53519
Z value for fiber 677: 0.53429
Warnings for fiber 675:
No warning
Warnings for fiber 677:
No warning

3929-55335-672
Z-difference: -9e-05
Z value for fiber 671: 0.4906
Z value for fiber 673: 0.49069
Warnings for fiber 671:
No warning
Warnings for fiber 673

In [2]:
g = ghosts.flux_data(3874,55280,68)

In [17]:
test_table = obj_data.select_all(what = 'PLATE,MJD,FIBER,Z,ZWARNING,OBJTYPE,CLASS,SUBCLASS',
                                 where = 'PLATE == 4077 and MJD == 55361 and FIBER between 311 and 313')

In [18]:
test_table

PLATE,MJD,FIBER,Z,ZWARNING,OBJTYPE,CLASS,SUBCLASS
int64,int64,int64,float64,int64,string128,string48,string112
4077,55361,311,0.45302,0,GALAXY,GALAXY,""""""
4077,55361,312,-0.00031412,0,SPECTROPHOTO_STD,STAR,F3/F5V+(30743)
4077,55361,313,0.45273,0,GALAXY,GALAXY,""""""
